# Multimodal Driver State Modeling through Unsupervised Learning

This repository is the code accompanying the paper with the same name. The data for this study is based on the HARMONY dataset. A sample of the data is available at https://osf.io/zextd/

In [ ]:
#-----------------------------------------------------------
#all packages
#-----------------------------------------------------------
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy
from scipy import stats
from scipy import signal
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
#All needed packages
from scipy.signal import find_peaks
from tqdm import tqdm
import os
from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture
import matplotlib.pyplot as plt
import re, nltk, spacy, gensim
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import kstest
from scipy.stats import chi2_contingency

###  Data from long trip for clustering

In [ ]:
#-----------------------------------------------------------
#2.1 Reading all sources of wearable, vehicle, and gaze data
#-----------------------------------------------------------

#read car's data
#reading acc,gyro, and linear acc data, put them together, resample at 100 hz, fill nans and make it a clean dataframe

sensors = ["GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]
df_ultimate = []

for sensor in sensors:
    df_temp = pd.read_csv("C:/Users/Arsalan/Desktop/CVILLE - DC Trips/both hands and car - DC - CVille/raw/car/"+"Smartwatch_"+sensor+".csv",parse_dates=['Timestamp'])
    #start_date = "2020-08-21T23:30:59.154+0000"
    #end_date = "2020-08-22T02:14:56.307+0000"
    start_date = "2020-08-23T15:16:11.000+0000"
    end_date = "2020-08-23T17:16:00.000+0000"
    mask = (df_temp['Timestamp'] > start_date) & (df_temp['Timestamp'] <= end_date)
    df_temp = df_temp.loc[mask]
    if sensor in ["GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]:
        df_temp.rename(columns={'X': 'X'+sensor[:4], 'Y': 'Y'+sensor[:4],'Z':'Z'+sensor[:4]}, inplace=True)
    df_ultimate.append(df_temp)

    
df_final = df_ultimate[0]
for df in df_ultimate[1:]:
    df = df.drop(columns=["participantid"])
    df.reset_index(drop=True,inplace=True)
    df_final = df_final.merge(df,on=['Timestamp',
                                         'DeviceId'], how='outer')
    
    
df_final = df_final.set_index("Timestamp")
df_final = df_final.groupby(["DeviceId"]).resample('100L').mean().ffill().bfill()
df_final = df_final.reset_index()
df_final["magGyro"] = (df_final["XGyro"]**2 + df_final["YGyro"]**2 + df_final["ZGyro"]**2)**0.5
df_final["magAcce"] = (df_final["XAcce"]**2 + df_final["YAcce"]**2 + df_final["ZAcce"]**2)**0.5
df_final["magLine"] = (df_final["XLine"]**2 + df_final["YLine"]**2 + df_final["ZLine"]**2)**0.5

#keep a copy to not rerun things
imu_data_car = df_final.copy()
imu_data_car = imu_data_car.set_index("Timestamp")
imu_data_car = df_final.copy()
imu_data_car = imu_data_car.set_index("Timestamp")
imu_data_car = imu_data_car[1:]

#2.2 Reading all sources of wearable, vehicle, and gaze data
#reading acc,gyro, and linear acc data, put them together, resample at 100 hz, fill nans and make it a clean dataframe


sensors = ["HeartRateDatum","GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum","Light"]
df_ultimate = []

for sensor in sensors:
    df_temp = pd.read_csv("C:/Users/Arsalan/Desktop/CVILLE - DC Trips/both hands and car - DC - CVille/raw/"+"Smartwatch_"+sensor+".csv",parse_dates=['Timestamp'])
#     start_date = "2020-08-21T23:30:59.154+0000"
#     end_date = "2020-08-22T02:14:56.307+0000"
    start_date = "2020-08-23T15:16:11.000+0000"
    end_date = "2020-08-23T17:16:00.000+0000"
    mask = (df_temp['Timestamp'] > start_date) & (df_temp['Timestamp'] <= end_date)
    df_temp = df_temp.loc[mask]
    if sensor in ["GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]:
        df_temp.rename(columns={'X': 'X'+sensor[:4]+"_driver", 'Y': 'Y'+sensor[:4]+"_driver",'Z':'Z'+sensor[:4]+"_driver"}, inplace=True)
    df_ultimate.append(df_temp)

    
df_final = df_ultimate[0]
for df in df_ultimate[1:]:
    df = df.drop(columns=["participantid"])
    df.reset_index(drop=True,inplace=True)
    df_final = df_final.merge(df,on=['Timestamp',
                                         'DeviceId'], how='outer')
    
    
df_final = df_final.set_index("Timestamp")
df_final = df_final.groupby(["DeviceId"]).resample('100L').mean().ffill().bfill()
df_final = df_final.reset_index()
df_final["magGyro_driver"] = (df_final["XGyro_driver"]**2 + df_final["YGyro_driver"]**2 + df_final["ZGyro_driver"]**2)**0.5
df_final["magAcce_driver"] = (df_final["XAcce_driver"]**2 + df_final["YAcce_driver"]**2 + df_final["ZAcce_driver"]**2)**0.5
df_final["magLine_driver"] = (df_final["XLine_driver"]**2 + df_final["YLine_driver"]**2 + df_final["ZLine_driver"]**2)**0.5

#keep a copy to not rerun things
hr_imu_data = df_final.copy()
hr_imu_data = hr_imu_data.set_index("Timestamp")
hr_imu_data_left=hr_imu_data[hr_imu_data["DeviceId"]=="f60691a313420a4e"]
hr_imu_data_right=hr_imu_data[hr_imu_data["DeviceId"]=="39ca51c16b9ec429"]
hr_imu_data_left.columns = [str(col) + '_left' for col in hr_imu_data_left.columns]
#imu_data_right = imu_data_right.set_index("Timestamp")
#imu_data_left = imu_data_left.set_index("Timestamp")
#2.3 Reading all sources of wearable, vehicle, and gaze data
#reading gaze data

cols = [ " AU01_r"," AU02_r"," AU04_r"," AU05_r"," AU06_r"," AU07_r"," AU09_r"," AU10_r"," AU12_r"," AU14_r"," AU15_r"," AU17_r"," AU20_r"," AU23_r"," AU25_r"," AU26_r"," AU45_r"," AU01_c"," AU02_c"," AU04_c"," AU05_c"," AU06_c"," AU07_c"," AU09_c"," AU10_c"," AU12_c"," AU14_c"," AU15_c"," AU17_c"," AU20_c"," AU23_c"," AU25_c"," AU26_c"," AU28_c", " AU45_c"
," timestamp"," confidence"," success"," gaze_angle_x"," gaze_angle_y"," p_scale"," p_rx"," p_ry"," p_rz"," p_tx"," p_ty","video_name","day" ]
gaze = pd.read_csv("C:/Users/Arsalan/Desktop/CVILLE - DC Trips/both hands and car - DC - CVille/raw/final_gaze.csv",usecols=cols)

gaze["time_start"] = (gaze["video_name"].str[:-7])
gaze["time_start"] = pd.to_datetime(gaze["time_start"],format = '%Y%m%d_%H%M%S')
gaze[" timestamp"] = pd.to_timedelta(gaze[" timestamp"],unit = 's')
gaze["Timestamp"] = gaze["time_start"] + gaze[" timestamp"]
gaze["mag_gaze"] = (gaze[" gaze_angle_x"]**2 + gaze[" gaze_angle_y"]**2)**0.5
gaze["Timestamp"] = gaze["Timestamp"] + pd.Timedelta(value=+4,unit='h')
# start_date = pd.Timestamp("2020-08-21T23:30:59.154+0000",tz=None)
# end_date = pd.Timestamp("2020-08-22T02:14:56.307+0000",tz=None)
start_date = pd.Timestamp("2020-08-23T15:16:11.000+0000",tz=None)
end_date = pd.Timestamp("2020-08-23T17:16:00.000+0000",tz=None)
start_date = start_date.tz_localize(None)
end_date = end_date.tz_localize(None)
gaze["Timestamp"] = pd.to_datetime(gaze["Timestamp"])  
gaze["Timestamp"] = gaze["Timestamp"].dt.tz_localize(None)
mask = (gaze["Timestamp"] > start_date) & (gaze["Timestamp"] <= end_date)
gaze = gaze.loc[mask]
gaze = gaze.set_index("Timestamp")
gaze = gaze[gaze[" success"] == 1]
gaze = gaze[gaze[" confidence"]>0.85]
gaze = gaze.resample('100L').mean().bfill().ffill()
gaze = gaze.drop(columns=['day'])

#2.4 getting all the outside environment data


cols = ["person","bicycle","car","motorcycle","bus","truck","frame","follow_dist","follow_obj","lead_pres","video_name","day" ]
mask_rcnn = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/Wearable Projects/Codes/Wearable_Event_detection/csv files/final_mask_rcnn.csv",usecols=cols)

mask_rcnn["time_start"] = (mask_rcnn["video_name"].str[:-16])
mask_rcnn["time_start"] = pd.to_datetime(mask_rcnn["time_start"],format = '%Y%m%d_%H%M%S')
mask_rcnn[" timestamp"] = mask_rcnn["frame"]*0.0333
mask_rcnn[" timestamp"] = pd.to_timedelta(mask_rcnn[" timestamp"],unit = 's')
mask_rcnn["Timestamp"] = mask_rcnn["time_start"] + mask_rcnn[" timestamp"]
mask_rcnn["Timestamp"] = mask_rcnn["Timestamp"] + pd.Timedelta(value=+4,unit='h')
# start_date = pd.Timestamp("2020-08-21T23:30:59.154+0000",tz=None)
# end_date = pd.Timestamp("2020-08-22T02:14:56.307+0000",tz=None)
start_date = pd.Timestamp("2020-08-23T15:16:11.000+0000",tz=None)
end_date = pd.Timestamp("2020-08-23T17:16:00.000+0000",tz=None)
start_date = start_date.tz_localize(None)
end_date = end_date.tz_localize(None)
mask_rcnn["Timestamp"] = pd.to_datetime(mask_rcnn["Timestamp"])  
mask_rcnn["Timestamp"] = mask_rcnn["Timestamp"].dt.tz_localize(None)
mask = (mask_rcnn["Timestamp"] > start_date) & (mask_rcnn["Timestamp"] <= end_date)
mask_rcnn = mask_rcnn.loc[mask]
mask_rcnn = mask_rcnn.set_index("Timestamp")
mask_rcnn["lead_pres"] = mask_rcnn["lead_pres"].fillna(0)
mask_rcnn["follow_obj"] = mask_rcnn["follow_obj"].fillna(15)
mask_rcnn["follow_dist"] = mask_rcnn["follow_dist"].fillna(5000)
mask_rcnn = mask_rcnn.resample('100L').bfill().ffill()

#3. put them all together

hr_imu_data_right.index = hr_imu_data_right.index.tz_localize(None)
hr_imu_data_left.index = hr_imu_data_left.index.tz_localize(None)
imu_data_car.index = imu_data_car.index.tz_localize(None)
total_data = pd.concat([mask_rcnn,gaze,hr_imu_data_right,hr_imu_data_left,imu_data_car],axis=1)

#4. combining them with change points - copying bcp groups from previous steps
total_data = total_data[:-1]
#total_data["bcp_groups"] = list_groups["new_group"].values
#total_data = total_data.dropna()
total_data["gaze_std_x"]=total_data[" gaze_angle_x"].rolling(10,min_periods=0).std()
total_data["gaze_std_x"] = total_data["gaze_std_x"].bfill()
total_data["gaze_std_y"]=total_data[" gaze_angle_y"].rolling(10,min_periods=0).std()
total_data["gaze_std_y"] = total_data["gaze_std_y"].bfill()
total_data = total_data.bfill().ffill()
#total_data.to_csv("data_coming_back_100L.csv")

In [ ]:
#-----------------------------------------------------------
#Preparing the speed data for change point and combining with total_data
#-----------------------------------------------------------

# You can skip if you have already done the change points 

speed_0823 = pd.read_csv("I:/Phase 1/9/Video/08232020/speed_csv/final_speed.csv")
speed_0823["Timestamp"] = speed_0823['Date'].str.cat(speed_0823['Time'],sep="T")
speed_0823["Timestamp"] = pd.to_datetime(speed_0823["Timestamp"], format="%m/%d/%YT%H:%M:%S")
speed_0823 = speed_0823.sort_values(by="Timestamp")
speed_0823["Timestamp"] = speed_0823["Timestamp"].dt.tz_localize(None)
speed_0823["Normalized Speed"] = speed_0823["Normalized Speed"].replace(to_replace="not available",method='bfill') 
speed_0823["Timestamp"] = speed_0823["Timestamp"] + pd.Timedelta(value=+4,unit='h')
start_date = pd.Timestamp("2020-08-23T15:16:11.000+0000",tz=None)
end_date = pd.Timestamp("2020-08-23T17:16:00.000+0000",tz=None)
start_date = start_date.tz_localize(None)
end_date = end_date.tz_localize(None)
mask = (speed_0823['Timestamp'] > start_date) & (speed_0823['Timestamp'] <= end_date)
speed_0823 = speed_0823.loc[mask]
data_for_bcp = total_data.resample('1S').mean().bfill().ffill()
data_for_bcp = data_for_bcp.reset_index()


data_0823_speed_hr = pd.merge(speed_0823,data_for_bcp,on=["Timestamp"])

data_0823_speed_hr.to_csv("cleaned_speed_0823.csv")

In [ ]:
#-----------------------------------------------------------
# ran bcp in R. Bring the output here for finding when HR and speed change point co-occur.
#-----------------------------------------------------------
# Then find the location of interest through peak detection

def find_any_peak(segment):    
    if ((segment > 0.1)).any():
        return 1
    else:
        return 0


speed_0823_bcp = pd.read_csv("cleaned_speed_0823_bcp.csv",parse_dates=["Timestamp"])


speed_0823_bcp["Timestamp"] = speed_0823_bcp["Timestamp"].dt.tz_localize(None)
#speed_0823_bcp["Timestamp"] = speed_0823_bcp["Timestamp"] - pd.Timedelta(hours=4)
speed_0823_bcp["diff"] = speed_0823_bcp["bcp_mean_HR"].diff()
speed_0823_bcp["diff"] = speed_0823_bcp["diff"].shift(-1)
speed_0823_bcp["type_of_cp"] = speed_0823_bcp["diff"].apply(lambda x: 1 if x>0 else 0)
speed_0823_bcp["binary_prob"] = 0
speed_0823_bcp["binary_prob"][(speed_0823_bcp["bcp_prob_HR"]!=0)&(speed_0823_bcp["type_of_cp"]==1)] = 1


speed_0823_bcp["HR_presence"] = speed_0823_bcp["binary_prob"].rolling(5).apply(find_any_peak).dropna()
speed_0823_bcp["speed_presence"] = speed_0823_bcp["bcp_prob_speed"].rolling(5).apply(find_any_peak).dropna()
speed_0823_bcp["peak_loc"] = speed_0823_bcp["HR_presence"] + speed_0823_bcp["speed_presence"]
speed_0823_bcp["peak_loc"][speed_0823_bcp["peak_loc"]!=2] =0
speed_0823_bcp["peak_loc"][speed_0823_bcp["peak_loc"]==2] =1


X = speed_0823_bcp["peak_loc"]
peaks, _ = find_peaks(X, distance=60)
boolean_obs = np.zeros(X.shape[0])
boolean_obs[peaks] = 1
speed_0823_bcp["boolean_peak"] = boolean_obs

list_of_segments = speed_0823_bcp["Timestamp"][speed_0823_bcp["boolean_peak"]==1].values

In [ ]:
#-----------------------------------------------------------
#save for later analysis of bcp of multivariate data
#-----------------------------------------------------------

total_data.to_csv("08232021_with_gmm_for_bcp.csv")

In [ ]:
#-----------------------------------------------------------
# Read the bcp results
#-----------------------------------------------------------

total_data = pd.read_csv("08232021_with_gmm_with_bcp.csv",parse_dates=["Timestamp"])

In [ ]:
#-----------------------------------------------------------
# Add the gaze data
#-----------------------------------------------------------

import numpy as np

max_x = total_data["gaze_angle_x"].max()
min_x = total_data["gaze_angle_x"].min()
max_y = total_data["gaze_angle_y"].max()
min_y = total_data["gaze_angle_y"].min()

def linspacing_the_range(x,y,min_x,min_y,max_x,max_y):
    XX = np.linspace(min_x, max_x, x)
    YY = np.linspace(min_y, max_y, y)
    return XX,YY

x=4
y=4
XX,YY = linspacing_the_range(x,y,min_x,min_y,max_x,max_y)

counter=0

for i in range(x-1):
    for j in range(y-1):
        total_data.loc[(total_data['gaze_angle_x'] >= XX[i]) & 
                 (total_data['gaze_angle_x'] <= XX[i+1])& 
                 (total_data['gaze_angle_y'] >= YY[j])&
                  (total_data['gaze_angle_y'] <= YY[j+1]), 'location'] = counter
        counter += 1

total_data['location'] = total_data['location'].astype(int)

from tqdm.notebook import tqdm
tqdm.pandas()

def transition_matrix(windowed_data):
    transitions = windowed_data.astype(int).values
    n = 1+ max(transitions) #number of states

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    m = np.array(M)
    m[m==0] = 1/(x*y)
    pA = m / m.sum()
    Shannon = -np.sum(pA*np.log2(pA))
    return Shannon

total_data["entropy_gaze"] = total_data["location"].rolling(600).progress_apply(transition_matrix)
total_data["entropy_gaze"] = total_data["entropy_gaze"].bfill()

In [ ]:
#-----------------------------------------------------------
#Exploring how many gmm components we need for transforming data to words - you can skip
#-----------------------------------------------------------

feats = ["ZGyro","XLine","YLine"]
X = total_data[feats].values
n_components = np.arange(1, 60)
models = [GaussianMixture(n, random_state=0).fit(X)
          for n in n_components]
plt.plot(n_components, [m.bic(X) for m in models], label='BIC')
plt.plot(n_components, [m.aic(X) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');
plt.show()

In [ ]:
#-----------------------------------------------------------
# perform gaussian mixture model on the data based on previous section's graph
# Accelerometer data
#-----------------------------------------------------------


feats = ["ZGyro","XLine","YLine"]
X = total_data[feats].values
labels = (GaussianMixture(n_components=50).fit(X)).predict(X)
total_data["acc_words"] = labels

#let's look at the distributions of GMM components
total_data["acc_words"].value_counts().plot(kind='bar')
total_data["acc_words"] = total_data["acc_words"]+100

In [ ]:
# perform gaussian mixture model on the data based on previous section's graph
# Gaze data

feats = ["entropy_gaze"]
X = total_data[feats].values
labels = (GaussianMixture(n_components=50).fit(X)).predict(X)
total_data["gaze_words"] = labels

#let's look at the distributions of GMM components
total_data["gaze_words"].value_counts().plot(kind='bar')
total_data["gaze_words"] = total_data["gaze_words"]+100

In [ ]:
#-----------------------------------------------------------
# perform gaussian mixture model on the data based on previous section's graph
# HR data
#-----------------------------------------------------------

feats = ["HR"]
X = total_data[feats].values
labels = (GaussianMixture(n_components=50).fit(X)).predict(X)
total_data["HR_words"] = labels

#let's look at the distributions of GMM components
total_data["HR_words"].value_counts().plot(kind='bar')
total_data["HR_words"] = total_data["HR_words"]+100

In [ ]:
#-----------------------------------------------------------
# find the segments of bcp through peak detection
#-----------------------------------------------------------

X = total_data["segment_bcp_prob"]
peaks, _ = find_peaks(X, distance=50)
#sns.distplot(peaks[1:]-peaks[:-1])
boolean_obs = np.zeros(X.shape[0])
boolean_obs[peaks] = 1
total_data["boolean_peak"] = boolean_obs

list_of_segments = total_data["Timestamp"][total_data["boolean_peak"]==1].values

total_data["group"] = total_data["boolean_peak"].ne(0).cumsum()
total_data["group"] = total_data["group"] + 1

In [ ]:
#-----------------------------------------------------------
#make documents out of segments for all three of acc, gaze, and HR
#-----------------------------------------------------------


def seg_to_list_acc(seg):
    out = ' '.join(seg['acc_words'].astype(str).tolist())
    return out
docs_acc = total_data.groupby(['group']).apply(seg_to_list_acc).tolist()

vectorizer = CountVectorizer()

data_vectorized_acc = vectorizer.fit_transform(docs_acc)


def seg_to_list_gaze(seg):
    out = ' '.join(seg['gaze_words'].astype(str).tolist())
    return out
docs_gaze = total_data.groupby(['group']).apply(seg_to_list_gaze).tolist()

data_vectorized_gaze = vectorizer.fit_transform(docs_gaze)

def seg_to_list_hr(seg):
    out = ' '.join(seg['HR_words'].astype(str).tolist())
    return out
docs_HR = total_data.groupby(['group']).apply(seg_to_list_hr).tolist()

data_vectorized_HR = vectorizer.fit_transform(docs_HR)

In [ ]:
#-----------------------------------------------------------
# Build LDA Model based on the previous section for acc
#-----------------------------------------------------------


lda_model_acc = LatentDirichletAllocation(n_components=4,  
                                      max_iter=10000,  
                                      learning_method='online',   
                                      random_state=100,          
                                      batch_size=128,            
                                      evaluate_every = -1, 
                                      n_jobs = -1 )
lda_output_acc = lda_model_acc.fit_transform(data_vectorized_acc)

# Create Document — Topic Matrix
best_lda_model = lda_model_acc
#lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ['Doc' + str(i) for i in range(len(docs_acc))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output_acc, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)

#Save behavioral clusters which are topic of documents

total_data["behavior_cluster_acc"] = "NAN"
for group in total_data["group"].unique():
    total_data["behavior_cluster_acc"][total_data["group"]==group] = df_document_topic["dominant_topic"][group-1]

In [ ]:
#-----------------------------------------------------------
# Build LDA Model based on the previous section for gaze
#-----------------------------------------------------------


lda_model_gaze = LatentDirichletAllocation(n_components=2,  
                                      max_iter=10000,  
                                      learning_method='online',   
                                      random_state=100,          
                                      batch_size=128,            
                                      evaluate_every = -1, 
                                      n_jobs = -1 )
lda_output_gaze = lda_model_gaze.fit_transform(data_vectorized_gaze)
best_lda_model = lda_model_gaze
#lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ['Doc' + str(i) for i in range(len(docs_gaze))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output_gaze, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)

#Save behavioral clusters which are topic of documents

total_data["behavior_cluster_gaze"] = "NAN"
for group in total_data["group"].unique():
    total_data["behavior_cluster_gaze"][total_data["group"]==group] = df_document_topic["dominant_topic"][group-1]

In [ ]:
#-----------------------------------------------------------
# Build LDA Model based on the previous section for hr
#-----------------------------------------------------------


lda_model_HR = LatentDirichletAllocation(n_components=2,  
                                      max_iter=10000,  
                                      learning_method='online',   
                                      random_state=100,          
                                      batch_size=128,            
                                      evaluate_every = -1, 
                                      n_jobs = -1 )
lda_output_HR = lda_model_HR.fit_transform(data_vectorized_HR)

# Create Document — Topic Matrix

best_lda_model = lda_model_HR
#lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ['Doc' + str(i) for i in range(len(docs_HR))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output_HR, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)

#Save behavioral clusters which are topic of documents

total_data["behavior_cluster_hr"] = "NAN"
for group in total_data["group"].unique():
    total_data["behavior_cluster_hr"][total_data["group"]==group] = df_document_topic["dominant_topic"][group-1]

In [ ]:
#-----------------------------------------------------------
#depending on the number of clusters, save either of them 
#-----------------------------------------------------------


total_data.to_csv("08232021_with_gmm_with_bcp_with_lda_2_cats.csv")

### Analysis is finished, below is plotting with speed data 

In [ ]:
#-----------------------------------------------------------
total_data_temp = pd.read_csv("08232021_with_gmm_with_bcp_with_lda_4_cats.csv",parse_dates=["Timestamp"])
total_data = pd.read_csv("08232021_with_gmm_with_bcp_with_lda_2_cats.csv",parse_dates=["Timestamp"])

In [ ]:
#-----------------------------------------------------------
#combine with 2 cat - just as a temporary item. You can skip later 
#-------------------------------------------------------------------------------

total_data["behavior_cluster_acc"] = total_data_temp["behavior_cluster_acc"]

In [ ]:
#-----------------------------------------------------------
#counting the number of normal/abnormal HR for testing in R - note that the HR clusters 
#are flipped and will be fixed in plotting - road curvature
#----------------------------------------------------------------------


group_road_curv = total_data[total_data["behavior_cluster_acc"]==2]
print("abnormal HR: ",group_road_curv.groupby('behavior_cluster_hr')["frame"].count()[0])
print("normal HR: ",group_road_curv.groupby('behavior_cluster_hr')["frame"].count()[1])

In [ ]:
#-----------------------------------------------------------
#counting the number of normal/abnormal HR for testing in R - note that the HR clusters 
#are flipped and will be fixed in plotting - Highway freeflow
#----------------------------------------------------------------------


group_road_curv = total_data[total_data["behavior_cluster_acc"]==3]
print("abnormal HR: ",group_road_curv.groupby('behavior_cluster_hr')["frame"].count()[0])
print("normal HR: ",group_road_curv.groupby('behavior_cluster_hr')["frame"].count()[1])

In [ ]:
#-----------------------------------------------------------
#counting the number of high/low gaze for testing in R - note that the gaze clusters 
#are flipped and will be fixed in plotting - Highway freeflow
#----------------------------------------------------------------------

group_road_curv = total_data[total_data["behavior_cluster_acc"]==3]
print("high GTE: ",group_road_curv.groupby('behavior_cluster_gaze')["frame"].count()[1])
print("low GTE: ",group_road_curv.groupby('behavior_cluster_gaze')["frame"].count()[0])

In [ ]:
#-----------------------------------------------------------
#performing kruskal test for comparing normal and harsh brake - HR
#-----------------------------------------------------------

group_0 = total_data[total_data["behavior_cluster_acc"]==0]
group_1 = total_data[total_data["behavior_cluster_acc"]==1]
stats.kruskal(group_0["behavior_cluster_hr"].values,group_1["behavior_cluster_hr"].values)

In [ ]:
#performing kruskal test for comparing normal and harsh brake - gaze
group_0 = total_data[total_data["behavior_cluster_acc"]==0]
group_1 = total_data[total_data["behavior_cluster_acc"]==1]
stats.kruskal(group_0["behavior_cluster_hr"].values,group_1["behavior_cluster_gaze"].values)

In [ ]:
#-----------------------------------------------------------
# combine with the speed data 
#----------------------------------------------------------------------------------------

speed_0823_bcp = pd.read_csv("cleaned_speed_0823_bcp.csv",parse_dates=["Timestamp"])
data_for_bcp_2 = total_data.copy()
data_for_bcp_2 = data_for_bcp_2.reset_index()
feats= ["behavior_cluster_acc","behavior_cluster_gaze","behavior_cluster_hr","entropy_gaze","Timestamp",'boolean_peak','segment_bcp_prob']
data_for_bcp_2 = data_for_bcp_2[feats]
data_for_bcp_2 = data_for_bcp_2.reset_index()
speed_0823_bcp = pd.merge(speed_0823_bcp,data_for_bcp_2,on=["Timestamp"])

speed_0823_bcp["behavior_cluster_acc_int"] = speed_0823_bcp["behavior_cluster_acc"].astype(int)
speed_0823_bcp["behavior_cluster_gaze_int"] = speed_0823_bcp["behavior_cluster_gaze"].astype(int)
speed_0823_bcp["behavior_cluster_hr_int"] = speed_0823_bcp["behavior_cluster_hr"].astype(int)

In [ ]:
#-----------------------------------------------------------
# Find statistics of each behavioral category
#-----------------------------------------------------------

stats = total_data.groupby(["behavior_cluster_acc"]).describe()
stats_states = speed_0823_bcp.groupby(["behavior_cluster_acc"]).describe()
stats_states.to_csv("stats_behaviors_patterns.csv")

In [ ]:
#-----------------------------------------------------------
# Find statistics of each state category
#-----------------------------------------------------------


stats = total_data.groupby(["behavior_cluster_hr"]).describe()
stats_states = speed_0823_bcp.groupby(["behavior_cluster_hr"]).describe()
stats_states.to_csv("stats_states_patterns_hr.csv")

stats = total_data.groupby(["behavior_cluster_gaze"]).describe()
stats_states = speed_0823_bcp.groupby(["behavior_cluster_gaze"]).describe()
stats_states.to_csv("stats_states_patterns_gaze.csv")

In [ ]:
#-----------------------------------------------------------
# testing differences between distributions
#-----------------------------------------------------------


beh_0 = speed_0823_bcp[speed_0823_bcp["behavior_cluster_hr"]==0]
beh_1 = speed_0823_bcp[speed_0823_bcp["behavior_cluster_hr"]==1]
beh_2 = speed_0823_bcp[speed_0823_bcp["behavior_cluster_hr"]==2]
beh_3 = speed_0823_bcp[speed_0823_bcp["behavior_cluster_hr"]==3]
w,p = kstest(beh_0["HR"],beh_1["HR"])
print(w,p)

In [ ]:
#-----------------------------------------------------------
#Plotting the fractions of each category of HR and gaze inside driving patterns - calculations
#---------------------------------------------------------------------------------------------------------------

speed_0823_bcp_temp_for_plot = speed_0823_bcp.copy()
speed_0823_bcp_temp_for_plot["behavior_cluster_hr"][speed_0823_bcp_temp_for_plot["behavior_cluster_hr"]==0] = 2
speed_0823_bcp_temp_for_plot["behavior_cluster_hr"][speed_0823_bcp_temp_for_plot["behavior_cluster_hr"]==1] = 0
speed_0823_bcp_temp_for_plot["behavior_cluster_hr"][speed_0823_bcp_temp_for_plot["behavior_cluster_hr"]==2] = 1

counted_data_hr = speed_0823_bcp_temp_for_plot.groupby(['behavior_cluster_acc',"behavior_cluster_hr"]).count()["frame"].reset_index()
counted_data_gaze = speed_0823_bcp_temp_for_plot.groupby(['behavior_cluster_acc',"behavior_cluster_gaze"]).count()["frame"].reset_index()
counted_data_hr["fraction"] = counted_data_hr.groupby(["behavior_cluster_acc"])["frame"].apply(lambda x: x/x.sum())
counted_data_gaze["fraction"] = counted_data_gaze.groupby(["behavior_cluster_acc"])["frame"].apply(lambda x: x/x.sum())


counted_data_hr["behavior_cluster_acc"][counted_data_hr["behavior_cluster_acc"]==0] = "Normal Brake"
counted_data_hr["behavior_cluster_acc"][counted_data_hr["behavior_cluster_acc"]==1] = "Harsh Brake"
counted_data_hr["behavior_cluster_acc"][counted_data_hr["behavior_cluster_acc"]==2] = "Road Curvature Driving"
counted_data_hr["behavior_cluster_acc"][counted_data_hr["behavior_cluster_acc"]==3] = "Free Flow Driving"
counted_data_hr["behavior_cluster_hr"][counted_data_hr["behavior_cluster_hr"]==0] = "Normal Heart Rate"
counted_data_hr["behavior_cluster_hr"][counted_data_hr["behavior_cluster_hr"]==1] = "Abnormal Heart Rate"


counted_data_gaze["behavior_cluster_acc"][counted_data_gaze["behavior_cluster_acc"]==0] = "Normal Brake"
counted_data_gaze["behavior_cluster_acc"][counted_data_gaze["behavior_cluster_acc"]==1] = "Harsh Brake"
counted_data_gaze["behavior_cluster_acc"][counted_data_gaze["behavior_cluster_acc"]==2] = "Road Curvature Driving"
counted_data_gaze["behavior_cluster_acc"][counted_data_gaze["behavior_cluster_acc"]==3] = "Free Flow Driving"
counted_data_gaze["behavior_cluster_gaze"][counted_data_gaze["behavior_cluster_gaze"]==0] = "Low Gaze Entropy"
counted_data_gaze["behavior_cluster_gaze"][counted_data_gaze["behavior_cluster_gaze"]==1] = "High Gaze Entropy"

In [ ]:
#-----------------------------------------------------------
#Plotting the fractions of each category of HR and gaze inside driving patterns - plotting
#---------------------------------------------------------------------------------------------------------------

fig = px.bar(counted_data_hr, x="behavior_cluster_acc", y="fraction",color="behavior_cluster_hr",pattern_shape="behavior_cluster_hr",pattern_shape_sequence=[".", "x"])
fig.update_layout(  
    {    
        'yaxis':{'title':"<b>Fraction of Each Pattern",'linecolor':'black'},
        'xaxis':{'title':"<b>Behavior Pattern",'linecolor':'black'},
        'height':600,
        'width':1000,
        'font':dict(size=20),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
        
    }
)
fig.update_layout(legend=dict(
    title="HR Pattern",
    yanchor="top",
    xanchor="left",
    orientation="h",
    x=0,
    y=1.15,
    font = dict(size=24)
    
))


fig.show()

In [ ]:
#-----------------------------------------------------------
#Plotting the fractions of each category of HR and gaze inside driving patterns - plotting
#---------------------------------------------------------------------------------------------------------------

fig = px.bar(counted_data_gaze, x="behavior_cluster_acc", y="fraction",color="behavior_cluster_gaze",
             pattern_shape="behavior_cluster_gaze",pattern_shape_sequence=[".", "x"]
            ,color_discrete_sequence=["red", "green"])
fig.update_layout(  
    {    
        'yaxis':{'title':"<b>Fraction of Each Pattern",'linecolor':'black'},
        'xaxis':{'title':"<b>Behavior Pattern",'linecolor':'black'},
        'height':600,
        'width':1000,
        'font':dict(size=20),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
        
    }
)
fig.update_layout(legend=dict(
    title="Gaze Pattern",
    yanchor="top",
    xanchor="left",
    orientation="h",
    x=0,
    y=1.15,
    font = dict(size=24)
    
))


fig.show()

In [ ]:
#-----------------------------------------------------------
#sample plot for the paper showing clusters and how they relate to real-world data
#------------------------------------------------------------------------------------

temp_plot = total_data.loc[30000:40000]
temp_plot["behavior_cluster_acc"] = pd.Categorical(temp_plot["behavior_cluster_acc"])

fig = px.scatter(
   x=temp_plot["Timestamp"],y = temp_plot["ZGyro"],color=temp_plot['behavior_cluster_acc']
)
fig.update_layout(  
    {    
        'yaxis':{'title':"<b>Lateral Angular Speed",'linecolor':'black'},
        'xaxis':{'title':"<b>Time (UTC)",'linecolor':'black'},
        'height':600,
        'width':1400,
        'font':dict(size=20),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)',
        'legend_title':'Behavioral Pattern'
        
    }
)
fig.update_layout(legend=dict(
    yanchor="bottom",
    xanchor="left",
    orientation="h",
    x=0,
    y=-0.25
    
))
fig.show()

fig = px.scatter(
   x=temp_plot["Timestamp"],y = temp_plot["YAcce"],color=temp_plot['behavior_cluster_acc']
)
fig.update_layout(  
    {    
        'yaxis':{'title':"<b>Forward Acceleration",'linecolor':'black'},
        'xaxis':{'title':"<b>Time (UTC)",'linecolor':'black'},
        'height':600,
        'width':1400,
        'font':dict(size=20),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)',
        'legend_title':'Behavioral Pattern'
        
    }
)
fig.update_layout(legend=dict(
    yanchor="bottom",
    xanchor="left",
    orientation="h",
    x=0,
    y=-0.25
    
))
fig.show()

fig = px.scatter(
   x=temp_plot["Timestamp"],y = temp_plot["XAcce"],color=temp_plot['behavior_cluster_acc']
)
fig.update_layout(  
    {    
        'yaxis':{'title':"<b>Lateral Acceleration",'linecolor':'black'},
        'xaxis':{'title':"<b>Time (UTC)",'linecolor':'black'},
        'height':600,
        'width':1400,
        'font':dict(size=20),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)',
        'legend_title':'Behavioral Pattern'
        
    }
)
fig.update_layout(legend=dict(
    yanchor="bottom",
    xanchor="left",
    orientation="h",
    x=0,
    y=-0.25
    
))
fig.show()

In [ ]:
#-----------------------------------------------------------
#plot state distributions within the acc patterns 
#------------------------------------------------------------------------------------

fig, axes = plt.subplots(1,3,figsize=(28,6))

axes[0].set_xlabel("Gaze Entropy",fontsize = 20)
axes[0].tick_params(axis='x', labelsize=20)
axes[0].set_ylabel("Density",fontsize = 20)
axes[0].tick_params(axis='y', labelsize=20)

axes[1].set_xlabel("HR",fontsize = 20)
axes[1].tick_params(axis='x', labelsize=20)
axes[1].set_ylabel("Density",fontsize = 20)
axes[1].tick_params(axis='y', labelsize=20)

axes[2].set_xlabel("Speed",fontsize = 20)
axes[2].tick_params(axis='x', labelsize=20)
axes[2].set_ylabel("Density",fontsize = 20)
axes[2].tick_params(axis='y', labelsize=20)
axes[2].set_xlim([0,125])



sns.kdeplot(
   data=speed_0823_bcp, x="entropy_gaze", hue="behavior_cluster_acc",ax=axes[0],palette="dark",linewidth=3
)
axes[0].get_legend().set_title("Behavioral Pattern")

sns.kdeplot(
   data=speed_0823_bcp, x="HR", hue="behavior_cluster_acc",linewidth=3,
  ax=axes[1],palette="dark"
)
axes[1].get_legend().set_title("Behavioral Pattern")

sns.kdeplot(
   data=speed_0823_bcp, x="Normalized.Speed", hue="behavior_cluster_acc",linewidth=3,
   ax=axes[2],palette="dark"
)
axes[2].get_legend().set_title("Behavioral Pattern")

#fig.savefig("distribution_1.png")

In [ ]:
#-----------------------------------------------------------
#plot kinematic distributions within the acc patterns 
#------------------------------------------------------------------------------------

fig, axes = plt.subplots(1,3,figsize=(28,6))

axes[0].set_xlabel("Forward Acceleration",fontsize = 20)
axes[0].tick_params(axis='x', labelsize=20)
axes[0].set_ylabel("Density",fontsize = 20)
axes[0].tick_params(axis='y', labelsize=20)
axes[0].set_xlim([-1.5,1.5])

axes[1].set_xlabel("Lateral Acceleration",fontsize = 20)
axes[1].tick_params(axis='x', labelsize=20)
axes[1].set_ylabel("Density",fontsize = 20)
axes[1].tick_params(axis='y', labelsize=20)
axes[1].set_xlim([-1.5,1.5])

axes[2].set_xlabel("Lateral Angular Speed",fontsize = 20)
axes[2].tick_params(axis='x', labelsize=20)
axes[2].set_ylabel("Density",fontsize = 20)
axes[2].tick_params(axis='y', labelsize=20)
axes[2].set_xlim([-0.055,0.055])




sns.kdeplot(
   data=speed_0823_bcp, x="YAcce", hue="behavior_cluster_acc",
   linewidth=5,ax=axes[0],palette="Spectral"
)
axes[0].get_legend().set_title("Behavioral Pattern")

sns.kdeplot(
   data=speed_0823_bcp, x="XAcce", hue="behavior_cluster_acc",
   linewidth=5,ax=axes[1],palette="Spectral"
)
axes[1].get_legend().set_title("Behavioral Pattern")

sns.kdeplot(
   data=speed_0823_bcp, x="ZGyro", hue="behavior_cluster_acc",
   linewidth=5,ax=axes[2],palette="Spectral"
)
axes[2].get_legend().set_title("Behavioral Pattern")

fig.savefig("distribution_2.png")

In [ ]:
#-----------------------------------------------------------
#plot all distributions during the acc patterns 
#------------------------------------------------------------------------------------

speed_0823_bcp_temp_for_plot = speed_0823_bcp.copy()
speed_0823_bcp_temp_for_plot["behavior_cluster_hr"][speed_0823_bcp_temp_for_plot["behavior_cluster_hr"]==0] = 2
speed_0823_bcp_temp_for_plot["behavior_cluster_hr"][speed_0823_bcp_temp_for_plot["behavior_cluster_hr"]==1] = 0
speed_0823_bcp_temp_for_plot["behavior_cluster_hr"][speed_0823_bcp_temp_for_plot["behavior_cluster_hr"]==2] = 1


fig, axes = plt.subplots(1,3,figsize=(28,6))

axes[0].set_xlabel("Gaze Entropy",fontsize = 20)
axes[0].tick_params(axis='x', labelsize=20)
axes[0].set_ylabel("Density",fontsize = 20)
axes[0].tick_params(axis='y', labelsize=20)

axes[1].set_xlabel("HR",fontsize = 20)
axes[1].tick_params(axis='x', labelsize=20)
axes[1].set_ylabel("Density",fontsize = 20)
axes[1].tick_params(axis='y', labelsize=20)

axes[2].set_xlabel("Speed",fontsize = 20)
axes[2].tick_params(axis='x', labelsize=20)
axes[2].set_ylabel("Density",fontsize = 20)
axes[2].tick_params(axis='y', labelsize=20)
axes[2].set_xlim([0,125])



sns.kdeplot(
   data=speed_0823_bcp_temp_for_plot, x="entropy_gaze", hue="behavior_cluster_gaze",ax=axes[0],palette="dark",linewidth=3
)
axes[0].get_legend().set_title("Gaze Entropy Pattern")

sns.kdeplot(
   data=speed_0823_bcp_temp_for_plot, x="HR", hue="behavior_cluster_hr",linewidth=3,
  ax=axes[1],palette="dark"
)
axes[1].get_legend().set_title("HR Pattern")

sns.kdeplot(
   data=speed_0823_bcp_temp_for_plot, x="Normalized.Speed", hue="behavior_cluster_hr",linewidth=3,
   ax=axes[2],palette="dark"
)
axes[2].get_legend().set_title("Behavioral Pattern")

fig.savefig("HR_gaze_cluster_2.png")

In [ ]:
counted_data_hr = speed_0823_bcp.groupby(['behavior_cluster_acc',"behavior_cluster_hr"]).count()["X"].reset_index()
counted_data_gaze = speed_0823_bcp.groupby(['behavior_cluster_acc',"behavior_cluster_gaze"]).count()["X"].reset_index()


counted_data_hr["behavior_cluster_acc"][counted_data_hr["behavior_cluster_acc"]==0] = "Normal Breaking"
counted_data_hr["behavior_cluster_acc"][counted_data_hr["behavior_cluster_acc"]==1] = "Harsh Breaking"
counted_data_hr["behavior_cluster_acc"][counted_data_hr["behavior_cluster_acc"]==2] = "Curve Driving"
counted_data_hr["behavior_cluster_acc"][counted_data_hr["behavior_cluster_acc"]==3] = "Free Flow Driving"
counted_data_hr["behavior_cluster_hr"][counted_data_hr["behavior_cluster_hr"]==0] = "Abnormal Heart Rate"
counted_data_hr["behavior_cluster_hr"][counted_data_hr["behavior_cluster_hr"]==1] = "Normal Heart Rate"


counted_data_gaze["behavior_cluster_acc"][counted_data_gaze["behavior_cluster_acc"]==0] = "Normal Breaking"
counted_data_gaze["behavior_cluster_acc"][counted_data_gaze["behavior_cluster_acc"]==1] = "Harsh Breaking"
counted_data_gaze["behavior_cluster_acc"][counted_data_gaze["behavior_cluster_acc"]==2] = "Curve Driving"
counted_data_gaze["behavior_cluster_acc"][counted_data_gaze["behavior_cluster_acc"]==3] = "Free Flow Driving"
counted_data_gaze["behavior_cluster_gaze"][counted_data_gaze["behavior_cluster_gaze"]==0] = "Low Gaze Entropy"
counted_data_gaze["behavior_cluster_gaze"][counted_data_gaze["behavior_cluster_gaze"]==1] = "High Gaze Entropy"

In [ ]:
#-----------------------------------------------------------
#Plot the behavioral clusters at each segments
#---------------------------------------------------------

#specific packages needed
#------------------------------------------------------------

import matplotlib as mpl
from matplotlib.colors import ListedColormap

speed_0823_bcp = speed_0823_bcp.set_index("Timestamp")
fig = plt.figure()
ax = speed_0823_bcp['Normalized.Speed'].plot()

def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)
#set_size(5,5)
plt.rcParams['font.size'] = '15'
plt.rcParams['font.family'] = 'sans-serif'
ax.figure.set_size_inches(19, 8)
ax.set_xlabel("Time",fontsize=25)
ax.set_ylabel("Speed(km/h)",fontsize=25)
ax.get_lines()[0].set_color("black")

cmap = ListedColormap(['gray', 'yellow','red','blue'])

z = ax.pcolorfast(ax.get_xlim(), ax.get_ylim(),
              speed_0823_bcp['behavior_cluster_acc_int'].values[np.newaxis],
              cmap=cmap,alpha=0.3)
plt.colorbar(z,ticks=[0,1,2,3,4])

fig.savefig("acc.png")

fig=plt.figure()
ax = speed_0823_bcp['Normalized.Speed'].plot()
def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)
#set_size(5,5)
plt.rcParams['font.size'] = '15'
plt.rcParams['font.family'] = 'sans-serif'
ax.figure.set_size_inches(19, 8)
ax.set_xlabel("Time",fontsize=25)
ax.set_ylabel("Speed(km/h)",fontsize=25)
ax.get_lines()[0].set_color("black")

cmap = ListedColormap(['gray','yellow'])
z = ax.pcolorfast(ax.get_xlim(), ax.get_ylim(),
              speed_0823_bcp['behavior_cluster_gaze_int'].values[np.newaxis],
              cmap=cmap, alpha=0.9)
plt.colorbar(z,ticks=[0,1])

fig.savefig("gaze.png")

fig=plt.figure()
ax = speed_0823_bcp['Normalized.Speed'].plot()
def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)
#set_size(5,5)
plt.rcParams['font.size'] = '15'
plt.rcParams['font.family'] = 'sans-serif'
ax.figure.set_size_inches(19, 8)
ax.set_xlabel("Time",fontsize=25)
ax.set_ylabel("Speed(km/h)",fontsize=25)
ax.get_lines()[0].set_color("black")

cmap = ListedColormap(['red','blue'])
z = ax.pcolorfast(ax.get_xlim(), ax.get_ylim(),
              speed_0823_bcp['behavior_cluster_hr_int'].values[np.newaxis],
              cmap=cmap, alpha=0.3)
plt.colorbar(z,ticks=[0,1,2,3,4])

fig.savefig("hr.png")



In [ ]:
#-----------------------------------------------------------
#Plot different segmentations for showcasing how segmentation works in the paper
#---------------------------------------------------------

# plotly line figure
fig = px.line(speed_0823_bcp, x=speed_0823_bcp["Timestamp"], y='Normalized.Speed')

# lines to add, specified by x-position
lines = list(speed_0823_bcp[speed_0823_bcp["segment_bcp_prob"]>0]["Timestamp"])

# add lines using absolute references
for k in range(len(lines)):
    #print(k)
    fig.add_shape(type='line',
                yref="y",
                xref="x",
                x0=lines[k],
                y0=speed_0823_bcp['Normalized.Speed'].min()*1.2,
                x1=lines[k],
                y1=speed_0823_bcp['Normalized.Speed'].max()*1.2,
                line=dict(color='red', width=2,dash='dot'))


    
fig.update_layout(  
    {    
        'yaxis':{'title':"<b>Speed (km/h)",'linecolor':'black'},
        'xaxis':{'title':"<b>Time (UTC)",'linecolor':'black'},
        'height':600,
        'width':1000,
        'font':dict(size=20),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
        
    }
)
fig.update_layout(legend=dict(
    yanchor="top",
    xanchor="left",
    
))
fig.update_traces(line_color='black')
fig.show()

### Calculate transition matrix

In [ ]:
#-----------------------------------------------------------
list_of_segments = total_data["Timestamp"][total_data["boolean_peak"]==1].values